<a href="https://colab.research.google.com/github/jalier/irpf/blob/main/Calculadora_Fiscal_IRPF_Espa%C3%B1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from collections import deque
import uuid # Para IDs únicos si es necesario

# Constantes para la regla antiaplicación
PERIODO_GENERAL_DIAS = 60 # 2 meses (aproximado)
PERIODO_EXTENDIDO_DIAS = 365 # 1 año

class Transaccion:
    def __init__(self, id_transaccion, asset_id, tipo_operacion, fecha_operacion, cantidad,
                 precio_unitario, divisa, comisiones, tipo_cambio_eur, broker=None):
        self.id_transaccion = id_transaccion # Identificador único para esta transacción
        self.asset_id = asset_id
        self.tipo_operacion = tipo_operacion.upper() # "COMPRA" o "VENTA"

        if isinstance(fecha_operacion, str):
            self.fecha_operacion = datetime.strptime(fecha_operacion, "%d/%m/%Y")
        else:
            self.fecha_operacion = fecha_operacion

        self.cantidad = float(cantidad)
        self.precio_unitario_original = float(precio_unitario)
        self.divisa = divisa
        self.comisiones_original = float(comisiones)

        if divisa == "EUR":
            self.tipo_cambio_eur = 1.0
        else:
            self.tipo_cambio_eur = float(tipo_cambio_eur) if tipo_cambio_eur else 1.0 # Asumir 1 si no se provee y no es EUR (o manejar error)

        self.broker = broker

        # Cálculos en EUR
        self.precio_unitario_eur = self.precio_unitario_original * self.tipo_cambio_eur
        self.comisiones_eur = self.comisiones_original * self.tipo_cambio_eur

        if self.tipo_operacion == "COMPRA":
            self.valor_adquisicion_unitario_eur_bruto = self.precio_unitario_eur
            self.valor_adquisicion_total_eur_bruto = self.cantidad * self.precio_unitario_eur
            self.costo_total_eur_con_comisiones = self.valor_adquisicion_total_eur_bruto + self.comisiones_eur
            self.precio_adquisicion_unitario_eur_neto = self.costo_total_eur_con_comisiones / self.cantidad if self.cantidad > 0 else 0
        elif self.tipo_operacion == "VENTA":
            self.valor_transmision_unitario_eur_bruto = self.precio_unitario_eur
            self.valor_transmision_total_eur_bruto = self.cantidad * self.precio_unitario_eur
            self.valor_transmision_total_eur_neto = self.valor_transmision_total_eur_bruto - self.comisiones_eur
            self.precio_transmision_unitario_eur_neto = self.valor_transmision_total_eur_neto / self.cantidad if self.cantidad > 0 else 0

    def __repr__(self):
        return (f"Transaccion({self.id_transaccion}, {self.asset_id}, {self.tipo_operacion}, "
                f"{self.fecha_operacion.strftime('%d/%m/%Y')}, {self.cantidad}, "
                f"{self.precio_unitario_original}, {self.divisa}, {self.comisiones_original}, "
                f"TC: {self.tipo_cambio_eur})")

class LoteCompra:
    def __init__(self, transaccion_compra):
        self.id_transaccion_origen = transaccion_compra.id_transaccion
        self.asset_id = transaccion_compra.asset_id
        self.fecha_compra = transaccion_compra.fecha_operacion
        self.cantidad_original = transaccion_compra.cantidad
        self.cantidad_restante = transaccion_compra.cantidad
        # Precio unitario de adquisición incluyendo comisiones prorrateadas
        self.precio_adquisicion_unitario_eur_neto = transaccion_compra.precio_adquisicion_unitario_eur_neto
        self.comisiones_compra_eur_proporcionales_inicial = transaccion_compra.comisiones_eur

        # Para la regla antiaplicación: si este lote bloquea una pérdida
        self.es_bloqueador = False
        self.ids_ventas_con_perdida_que_bloquea = [] # Lista de IDs de transacciones de venta cuya pérdida es bloqueada por este lote

    def vender_cantidad(self, cantidad_a_vender):
        if cantidad_a_vender > self.cantidad_restante:
            raise ValueError("No se puede vender más cantidad de la restante en el lote.")
        self.cantidad_restante -= cantidad_a_vender
        costo_vendido = cantidad_a_vender * self.precio_adquisicion_unitario_eur_neto
        return costo_vendido

    def __repr__(self):
        return (f"LoteCompra({self.id_transaccion_origen}, {self.asset_id}, {self.fecha_compra.strftime('%d/%m/%Y')}, "
                f"Orig: {self.cantidad_original}, Rest: {self.cantidad_restante}, "
                f"PrecioNetoEUR: {self.precio_adquisicion_unitario_eur_neto:.2f})")

class ResultadoVenta:
    def __init__(self, transaccion_venta, valor_adquisicion_eur, lotes_usados_info):
        self.transaccion_venta = transaccion_venta
        self.asset_id = transaccion_venta.asset_id
        self.fecha_venta = transaccion_venta.fecha_operacion
        self.cantidad_vendida = transaccion_venta.cantidad
        self.valor_transmision_eur_neto = transaccion_venta.valor_transmision_total_eur_neto
        self.valor_adquisicion_eur_neto = valor_adquisicion_eur # Suma de costos de los lotes FIFO
        self.ganancia_perdida_eur = self.valor_transmision_eur_neto - self.valor_adquisicion_eur_neto
        self.lotes_usados_info = lotes_usados_info # Lista de tuplas (id_lote_compra, cantidad_usada_de_lote, costo_de_esa_cantidad)

        self.es_perdida_bloqueada = False
        self.id_perdida_bloqueada_asociada = None # ID del registro de PerdidaBloqueada
        self.fiscal_year = self.fecha_venta.year

    def __repr__(self):
        status = "BLOQUEADA" if self.es_perdida_bloqueada else "NO BLOQUEADA"
        return (f"ResultadoVenta({self.transaccion_venta.id_transaccion}, {self.asset_id}, {self.fecha_venta.strftime('%d/%m/%Y')}, "
                f"G/P: {self.ganancia_perdida_eur:.2f} EUR, Status: {status})")

class PerdidaBloqueada:
    def __init__(self, id_perdida, resultado_venta_original, ids_compras_bloqueantes, regla_aplicada_dias):
        self.id_perdida = id_perdida # UUID
        self.asset_id = resultado_venta_original.asset_id
        self.id_transaccion_venta_original = resultado_venta_original.transaccion_venta.id_transaccion
        self.fecha_venta_original = resultado_venta_original.fecha_venta
        self.importe_perdida_bloqueada_eur = resultado_venta_original.ganancia_perdida_eur # Será negativo
        self.fiscal_year_generacion = resultado_venta_original.fiscal_year

        self.ids_transacciones_compras_bloqueantes = ids_compras_bloqueantes # Lista de IDs de transacciones de compra
        self.regla_aplicada_dias = regla_aplicada_dias # 60 o 365

        self.status = "ACTIVA" # Puede ser "ACTIVA" o "DESBLOQUEADA"
        self.fecha_desbloqueo = None
        self.fiscal_year_desbloqueo = None

    def __repr__(self):
        return (f"PerdidaBloqueada({self.id_perdida}, Asset: {self.asset_id}, "
                f"Venta: {self.id_transaccion_venta_original} ({self.fecha_venta_original.strftime('%d/%m/%Y')}), "
                f"Importe: {self.importe_perdida_bloqueada_eur:.2f} EUR, Status: {self.status}, "
                f"BloqPor: {self.ids_transacciones_compras_bloqueantes})")

class CalculadoraFiscal:
    def __init__(self, transacciones_historicas, config_cripto_periodo_largo=False):
        # Asignar IDs únicos a las transacciones si no los tienen
        for i, t in enumerate(transacciones_historicas):
            if not hasattr(t, 'id_transaccion') or t.id_transaccion is None:
                t.id_transaccion = str(uuid.uuid4())

        self.transacciones_historicas = sorted(transacciones_historicas, key=lambda t: t.fecha_operacion)
        self.config_cripto_periodo_largo = config_cripto_periodo_largo # True si cripto usa 1 año, False si usa 2 meses

        self.lotes_compra_activos = {} # asset_id -> deque[LoteCompra]
        self.resultados_ventas_por_asset = {} # asset_id -> list[ResultadoVenta]
        self.perdidas_bloqueadas_activas = [] # Lista global de PerdidaBloqueada con status ACTIVA
        self.perdidas_desbloqueadas_historial = [] # Lista global de PerdidaBloqueada con status DESBLOQUEADA

    def _get_periodo_antiaplicacion_dias(self, asset_id):
        # Esta lógica puede necesitar ser más sofisticada para determinar si un activo es cotizado o no.
        # Por ahora, asumimos que las criptomonedas pueden tener una regla diferente.
        # Aquí se podría verificar el asset_id contra una lista de mercados regulados.
        # Ejemplo simple:
        if "BTC" in asset_id.upper() or "ETH" in asset_id.upper(): # Asumir que son criptos
            return PERIODO_EXTENDIDO_DIAS if self.config_cripto_periodo_largo else PERIODO_GENERAL_DIAS
        # Para acciones, por defecto 2 meses, a menos que se sepa que no cotizan en mercado regulado.
        # Esta parte es crucial y depende de la naturaleza del activo.
        # Aquí se podría añadir una lógica para que el usuario especifique por AssetID si es mercado no regulado.
        # Por ahora, usamos PERIODO_GENERAL_DIAS como default para no-criptos.
        return PERIODO_GENERAL_DIAS


    def procesar_transacciones(self):
        for asset_id in set(t.asset_id for t in self.transacciones_historicas):
            self.lotes_compra_activos[asset_id] = deque()
            self.resultados_ventas_por_asset[asset_id] = []

        # Primera pasada: procesar compras y ventas, aplicar FIFO, identificar pérdidas y aplicar bloqueo inicial.
        for transaccion in self.transacciones_historicas:
            asset_id = transaccion.asset_id

            # Antes de procesar la transacción actual, verificar si esta venta desbloquea pérdidas anteriores
            # Esto es complejo porque el desbloqueo depende de la venta de lotes *específicos* que causaron el bloqueo.
            # Esta verificación se hará después de procesar la venta y saber qué lotes se usaron.

            if transaccion.tipo_operacion == "COMPRA":
                lote = LoteCompra(transaccion)
                self.lotes_compra_activos[asset_id].append(lote)

            elif transaccion.tipo_operacion == "VENTA":
                cantidad_a_vender = transaccion.cantidad
                costo_adquisicion_total_eur = 0
                lotes_usados_para_esta_venta_info = [] # (id_lote_compra, cantidad_usada, costo_de_esa_cantidad)

                # IDs de los lotes de compra originales que se consumen en esta venta
                ids_lotes_compra_consumidos_en_esta_venta = []

                if not self.lotes_compra_activos.get(asset_id):
                    print(f"ADVERTENCIA: Venta de {asset_id} el {transaccion.fecha_operacion.strftime('%d/%m/%Y')} sin lotes de compra previos registrados. Se asume costo 0.")
                    # Esto podría ser un error o una situación de datos incompletos.
                    # Para el cálculo, se registraría una ganancia total igual al valor de transmisión.
                    # O se podría lanzar un error más estricto.
                    # Por ahora, se procesa con costo 0, lo que maximiza la ganancia (o minimiza la pérdida).

                cantidad_vendida_acumulada_para_esta_transaccion = 0
                temp_lotes_usados_detalle = [] # [(LoteCompra_obj, cantidad_tomada_de_este_lote)]

                while cantidad_vendida_acumulada_para_esta_transaccion < cantidad_a_vender and self.lotes_compra_activos.get(asset_id):
                    lote_actual_fifo = self.lotes_compra_activos[asset_id][0]
                    cantidad_a_tomar_de_lote_actual = min(cantidad_a_vender - cantidad_vendida_acumulada_para_esta_transaccion, lote_actual_fifo.cantidad_restante)

                    costo_de_esta_porcion = lote_actual_fifo.precio_adquisicion_unitario_eur_neto * cantidad_a_tomar_de_lote_actual
                    costo_adquisicion_total_eur += costo_de_esta_porcion

                    lotes_usados_para_esta_venta_info.append((lote_actual_fifo.id_transaccion_origen, cantidad_a_tomar_de_lote_actual, costo_de_esta_porcion))
                    ids_lotes_compra_consumidos_en_esta_venta.append(lote_actual_fifo.id_transaccion_origen)
                    temp_lotes_usados_detalle.append((lote_actual_fifo, cantidad_a_tomar_de_lote_actual))

                    lote_actual_fifo.cantidad_restante -= cantidad_a_tomar_de_lote_actual
                    cantidad_vendida_acumulada_para_esta_transaccion += cantidad_a_tomar_de_lote_actual

                    if lote_actual_fifo.cantidad_restante == 0:
                        self.lotes_compra_activos[asset_id].popleft()

                if cantidad_vendida_acumulada_para_esta_transaccion < cantidad_a_vender:
                    # Esto significa que se vendió más de lo que había en cartera según los registros.
                    # Puede ser un error de datos o una venta en corto no soportada por este modelo simple.
                    print(f"ADVERTENCIA: Venta de {transaccion.cantidad} de {asset_id} el {transaccion.fecha_operacion.strftime('%d/%m/%Y')} pero solo se encontraron {cantidad_vendida_acumulada_para_esta_transaccion} unidades en cartera. El cálculo de la pérdida/ganancia se basará en las unidades encontradas.")
                    # Aquí se podría ajustar transaccion.cantidad a cantidad_vendida_acumulada_para_esta_transaccion para el cálculo de ResultadoVenta
                    # O manejar como un error crítico. Por ahora, se continúa con la cantidad que se pudo justificar.

                resultado_venta = ResultadoVenta(transaccion, costo_adquisicion_total_eur, lotes_usados_para_esta_venta_info)
                self.resultados_ventas_por_asset.setdefault(asset_id, []).append(resultado_venta)

                # Lógica de antiaplicación si es una pérdida
                if resultado_venta.ganancia_perdida_eur < 0:
                    periodo_dias = self._get_periodo_antiaplicacion_dias(asset_id)
                    fecha_venta = resultado_venta.fecha_venta
                    limite_anterior = fecha_venta - timedelta(days=periodo_dias)
                    limite_posterior = fecha_venta + timedelta(days=periodo_dias)

                    compras_bloqueantes_ids = []

                    # Buscar recompras (posteriores a la venta con pérdida, o anteriores pero aún en cartera)
                    # Primero, compras posteriores dentro del plazo:
                    for t_futura in self.transacciones_historicas: # Podría optimizarse buscando solo en el rango de fechas
                        if t_futura.asset_id == asset_id and \
                           t_futura.tipo_operacion == "COMPRA" and \
                           t_futura.fecha_operacion > fecha_venta and \
                           t_futura.fecha_operacion <= limite_posterior:
                            compras_bloqueantes_ids.append(t_futura.id_transaccion)
                            # Marcar el lote de esta compra como bloqueador potencial
                            # Esto requiere encontrar el LoteCompra correspondiente a t_futura.id_transaccion
                            # Esta marcación se hará mejor en una pasada posterior o referenciando por ID.

                    # Segundo, compras anteriores dentro del plazo cuyas acciones aún están en cartera (total o parcialmente)
                    # "En cartera" significa que el LoteCompra original todavía tiene cantidad_restante > 0
                    # O, más simple, si la compra original (t_pasada) ocurrió y no todas sus acciones fueron vendidas *antes* de la venta actual.
                    for lote_activo in self.lotes_compra_activos[asset_id]: # Lotes que aún tienen existencias
                        if lote_activo.fecha_compra >= limite_anterior and lote_activo.fecha_compra < fecha_venta:
                             # Si este lote_activo (que es una compra anterior a la venta actual)
                             # está dentro del periodo de 2 meses/1 año ANTES de la venta,
                             # y AÚN tiene acciones, entonces es un candidato a ser bloqueante.
                             # La normativa es compleja aquí: "se hubieran adquirido valores homogéneos".
                             # Si se compró antes y se mantiene, y se vende con pérdida, y se vuelve a comprar después, la pérdida se bloquea.
                             # Si se compró antes, se vende con pérdida, y NO se recompra después, la pérdida NO se bloquea por la compra anterior.
                             # El bloqueo se produce por recompra. La compra anterior es relevante si hay recompra posterior.
                             # La redacción "dos meses antes o después" sugiere que una compra anterior puede ser relevante
                             # si la venta con pérdida se produce, y luego hay una recompra que "refuerza" el bloqueo.
                             # La interpretación más común es que la pérdida se bloquea si hay recompra (posterior o anterior si aún se mantiene y se recompra más).
                             # Para simplificar y seguir la pauta "si se detecta una compra de valores homogéneos dentro del plazo relevante":
                             # Nos enfocamos en las recompras (posteriores) o compras (anteriores) que "rodean" la venta.
                             pass # La lógica de compras anteriores es más compleja y a menudo se enfoca en si se "sustituyen" los valores.
                                  # La AEAT suele centrarse en recompras posteriores, o si se compró antes, se vendió, y se compró de nuevo.
                                  # Por ahora, nos centraremos en las recompras posteriores como principal causa de bloqueo,
                                  # y las compras anteriores si son MUY cercanas y se mantienen.
                                  # La redacción "adquirido [...] dos meses antes [...] o dos meses después"
                                  # implica que una compra anterior (si aún se tienen esos títulos) Y una venta con pérdida,
                                  # seguida de una recompra posterior, o simplemente la existencia de esa compra anterior
                                  # en el periodo, puede ser suficiente.

                    # Simplificación: el bloqueo se produce si hay recompra en los X días ANTES o DESPUÉS.
                    # Y las acciones recompradas (o las compradas antes y aún mantenidas) son las que deben venderse.
                    # Vamos a buscar todas las compras en la ventana [-X, +X] días alrededor de la venta.

                    # Reiniciamos compras_bloqueantes_ids para una lógica más clara de ventana completa
                    compras_bloqueantes_ids = []
                    compras_candidatas_a_bloquear = []

                    for t_otra in self.transacciones_historicas:
                        if t_otra.asset_id == asset_id and t_otra.tipo_operacion == "COMPRA":
                            if (fecha_venta - timedelta(days=periodo_dias) <= t_otra.fecha_operacion < fecha_venta) or \
                               (fecha_venta < t_otra.fecha_operacion <= fecha_venta + timedelta(days=periodo_dias)):
                                compras_candidatas_a_bloquear.append(t_otra)

                    # De estas candidatas, ¿cuáles realmente bloquean?
                    # Si hay *alguna* compra en esa ventana, la pérdida se bloquea.
                    # Y se asocia a *todas* esas compras.
                    if compras_candidatas_a_bloquear:
                        ids_reales_bloqueantes = [t_compra.id_transaccion for t_compra in compras_candidatas_a_bloquear]

                        # Crear el registro de pérdida bloqueada
                        id_nueva_perdida_bloqueada = str(uuid.uuid4())
                        nueva_perdida = PerdidaBloqueada(id_nueva_perdida_bloqueada, resultado_venta, ids_reales_bloqueantes, periodo_dias)
                        self.perdidas_bloqueadas_activas.append(nueva_perdida)

                        resultado_venta.es_perdida_bloqueada = True
                        resultado_venta.id_perdida_bloqueada_asociada = id_nueva_perdida_bloqueada

                        # Marcar los lotes de compra correspondientes a 'ids_reales_bloqueantes' como 'es_bloqueador'
                        # Esto es para referencia, la lógica de desbloqueo usará PerdidaBloqueada.ids_transacciones_compras_bloqueantes
                        for id_compra_bloq in ids_reales_bloqueantes:
                            for lote_q in self.lotes_compra_activos.get(asset_id, deque()): # Lotes aún en cartera
                                if lote_q.id_transaccion_origen == id_compra_bloq:
                                    lote_q.es_bloqueador = True
                                    if resultado_venta.transaccion_venta.id_transaccion not in lote_q.ids_ventas_con_perdida_que_bloquea:
                                        lote_q.ids_ventas_con_perdida_que_bloquea.append(resultado_venta.transaccion_venta.id_transaccion)
                            # También podría aplicar a lotes ya consumidos si la compra fue anterior y ya se vendió parte.
                            # La lógica de "completamente vendidos" se verificará contra los IDs originales de las transacciones de compra.

                # Después de procesar la venta y registrarla (y potencialmente bloquearla):
                # Verificar si esta venta (al consumir ciertos lotes de compra) desbloquea alguna pérdida activa.
                # Una pérdida se desbloquea si TODOS los lotes de compra que la estaban bloqueando han sido COMPLETAMENTE vendidos.

                # `ids_lotes_compra_consumidos_en_esta_venta` tiene los IDs de las *transacciones de compra originales*
                # cuyos lotes fueron (parcial o totalmente) consumidos por la venta actual.

                # Necesitamos saber el estado de todos los lotes de compra (cantidad restante).
                # Esto se puede obtener creando un "snapshot" del estado de todos los lotes (originales)
                # o verificando uno por uno.

                # Iterar sobre las pérdidas activas para ver si alguna se desbloquea
                for perdida_activa in list(self.perdidas_bloqueadas_activas): # Iterar sobre una copia si se modifica la lista
                    if perdida_activa.status == "ACTIVA":
                        todas_las_compras_bloqueantes_vendidas = True
                        for id_compra_bloqueante in perdida_activa.ids_transacciones_compras_bloqueantes:
                            # ¿Ha sido esta compra bloqueante completamente vendida?
                            # Necesitamos encontrar la transacción de compra original y luego ver si todos sus lotes FIFO se han ido.
                            # Esto es más fácil si rastreamos la cantidad restante de cada lote original.

                            # Encuentra el LoteCompra original (si aún existe en lotes_compra_activos)
                            lote_original_de_compra_bloqueante_encontrado_en_activos = False
                            for lote_en_cartera in self.lotes_compra_activos.get(perdida_activa.asset_id, deque()):
                                if lote_en_cartera.id_transaccion_origen == id_compra_bloqueante:
                                    if lote_en_cartera.cantidad_restante > 0:
                                        todas_las_compras_bloqueantes_vendidas = False # Aún queda algo de este lote bloqueante
                                    lote_original_de_compra_bloqueante_encontrado_en_activos = True
                                    break

                            if not lote_original_de_compra_bloqueante_encontrado_en_activos and todas_las_compras_bloqueantes_vendidas:
                                # Si no está en lotes activos, significa que se consumió completamente en el pasado o en esta venta.
                                # (Asumiendo que los lotes se quitan de lotes_compra_activos[asset_id] cuando cantidad_restante es 0)
                                pass # Se considera vendido
                            elif not todas_las_compras_bloqueantes_vendidas:
                                break # Uno de los bloqueantes aún no está completamente vendido

                        if todas_las_compras_bloqueantes_vendidas:
                            perdida_activa.status = "DESBLOQUEADA"
                            perdida_activa.fecha_desbloqueo = transaccion.fecha_operacion # Fecha de la venta que lo desbloquea
                            perdida_activa.fiscal_year_desbloqueo = transaccion.fecha_operacion.year
                            self.perdidas_desbloqueadas_historial.append(perdida_activa)
                            self.perdidas_bloqueadas_activas.remove(perdida_activa)


    def generar_resumen_fiscal_anual(self, year):
        resumen = {
            "year": year,
            "total_ganancias_brutas_ventas_del_ano": 0,
            "total_perdidas_brutas_ventas_del_ano": 0, # Antes de antiaplicación
            "total_perdidas_bloqueadas_generadas_en_el_ano": 0,
            "total_perdidas_desbloqueadas_en_el_ano_de_anos_anteriores": 0,
            "total_perdidas_desbloqueadas_en_el_ano_generadas_en_el_mismo_ano":0,
            "saldo_neto_ganancias_perdidas_patrimoniales_imputables": 0,
            "perdidas_pendientes_compensar_ejercicios_siguientes": [] # Lista de (año_origen, importe_pendiente)
        }

        # Ganancias y pérdidas de ventas realizadas en 'year'
        for asset_id, ventas_asset in self.resultados_ventas_por_asset.items():
            for venta in ventas_asset:
                if venta.fiscal_year == year:
                    if venta.ganancia_perdida_eur >= 0:
                        resumen["total_ganancias_brutas_ventas_del_ano"] += venta.ganancia_perdida_eur
                    else: # Es una pérdida
                        resumen["total_perdidas_brutas_ventas_del_ano"] += venta.ganancia_perdida_eur # Sumar el negativo

                        if venta.es_perdida_bloqueada:
                            # ¿Esta pérdida fue bloqueada en el mismo año que se generó?
                            pb_asociada = next((pb for pb in self.perdidas_bloqueadas_activas + self.perdidas_desbloqueadas_historial if pb.id_perdida == venta.id_perdida_bloqueada_asociada), None)
                            if pb_asociada and pb_asociada.fiscal_year_generacion == year :
                                resumen["total_perdidas_bloqueadas_generadas_en_el_ano"] += venta.ganancia_perdida_eur # Sumar el negativo

        # Pérdidas desbloqueadas en 'year'
        perdidas_imputables_este_ano = resumen["total_ganancias_brutas_ventas_del_ano"] # Empezamos con las ganancias

        # Sumar pérdidas no bloqueadas del año actual
        for asset_id, ventas_asset in self.resultados_ventas_por_asset.items():
            for venta in ventas_asset:
                if venta.fiscal_year == year and venta.ganancia_perdida_eur < 0 and not venta.es_perdida_bloqueada:
                    perdidas_imputables_este_ano += venta.ganancia_perdida_eur

        # Sumar pérdidas desbloqueadas en este año (pueden ser de este año o anteriores)
        for pd_desbloq in self.perdidas_desbloqueadas_historial:
            if pd_desbloq.fiscal_year_desbloqueo == year:
                # Esta pérdida se suma al saldo del año de desbloqueo.
                perdidas_imputables_este_ano += pd_desbloq.importe_perdida_bloqueada_eur # Es un valor negativo
                if pd_desbloq.fiscal_year_generacion < year:
                    resumen["total_perdidas_desbloqueadas_en_el_ano_de_anos_anteriores"] += pd_desbloq.importe_perdida_bloqueada_eur
                else: # fiscal_year_generacion == year
                    resumen["total_perdidas_desbloqueadas_en_el_ano_generadas_en_el_mismo_ano"] += pd_desbloq.importe_perdida_bloqueada_eur


        resumen["saldo_neto_ganancias_perdidas_patrimoniales_imputables"] = perdidas_imputables_este_ano

        # Lógica de compensación de pérdidas de años anteriores (simplificada aquí)
        # Esto requeriría llevar un control más detallado de las pérdidas pendientes por año de origen.
        # Por ahora, el saldo neto es lo que se integraría. Si es negativo, se convierte en pérdida a compensar.

        if resumen["saldo_neto_ganancias_perdidas_patrimoniales_imputables"] < 0:
            resumen["perdidas_pendientes_compensar_ejercicios_siguientes"].append({
                "ano_generacion_saldo_negativo": year,
                "importe_pendiente": resumen["saldo_neto_ganancias_perdidas_patrimoniales_imputables"],
                "limite_compensacion_ano": year + 4
            })

        # Aquí se debería añadir lógica para usar saldos negativos de años anteriores contra ganancias actuales,
        # respetando el límite de 4 años.

        return resumen

    def generar_informe_detallado_por_activo(self, asset_id_objetivo):
        informe = []
        # Reconstruir cronología para el activo
        transacciones_activo = sorted([t for t in self.transacciones_historicas if t.asset_id == asset_id_objetivo], key=lambda t: t.fecha_operacion)

        # Necesitamos recalcular FIFO "en vivo" para este informe o usar los resultados ya procesados
        # de una manera que muestre el balance.
        # Por simplicidad, mostramos las transacciones y los resultados de venta asociados.

        # Lotes de compra para este activo (estado final)
        # print(f"\nLotes de compra activos para {asset_id_objetivo} (estado final):")
        # for lote in self.lotes_compra_activos.get(asset_id_objetivo, []):
        #     print(lote)

        # print(f"\nResultados de ventas para {asset_id_objetivo}:")
        # for venta_res in self.resultados_ventas_por_asset.get(asset_id_objetivo, []):
        #     print(venta_res)
        #     if venta_res.es_perdida_bloqueada:
        #         pb_info = next((pb for pb in self.perdidas_bloqueadas_activas + self.perdidas_desbloqueadas_historial if pb.id_perdida == venta_res.id_perdida_bloqueada_asociada),None)
        #         print(f"  Bloqueada por: {pb_info.ids_transacciones_compras_bloqueantes if pb_info else 'Error ID no encontrado'}")
        #     print(f"  Lotes FIFO usados: {venta_res.lotes_usados_info}")

        # Esta función de informe detallado necesita más trabajo para ser realmente útil y mostrar el FIFO paso a paso.
        # Por ahora, devolvemos las transacciones y los resultados de venta calculados.

        for t in transacciones_activo:
            detalle_t = {
                "ID Transaccion": t.id_transaccion,
                "Fecha": t.fecha_operacion.strftime('%d/%m/%Y'),
                "Tipo": t.tipo_operacion,
                "Cantidad": t.cantidad,
                "Precio Unitario Orig.": t.precio_unitario_original,
                "Divisa": t.divisa,
                "Comisiones Orig.": t.comisiones_original,
                "Tipo Cambio EUR": t.tipo_cambio_eur,
                "Broker": t.broker,
            }
            if t.tipo_operacion == "VENTA":
                venta_res = next((vr for vr in self.resultados_ventas_por_asset.get(asset_id_objetivo, []) if vr.transaccion_venta.id_transaccion == t.id_transaccion), None)
                if venta_res:
                    detalle_t["Valor Adquisicion EUR (FIFO)"] = f"{venta_res.valor_adquisicion_eur_neto:.2f}"
                    detalle_t["Valor Transmision EUR Neto"] = f"{venta_res.valor_transmision_eur_neto:.2f}"
                    detalle_t["Ganancia/Perdida EUR"] = f"{venta_res.ganancia_perdida_eur:.2f}"
                    detalle_t["Perdida Bloqueada"] = venta_res.es_perdida_bloqueada
                    if venta_res.es_perdida_bloqueada:
                         pb_info = next((pb for pb in self.perdidas_bloqueadas_activas + self.perdidas_desbloqueadas_historial if pb.id_perdida == venta_res.id_perdida_bloqueada_asociada),None)
                         detalle_t["Causa Bloqueo (IDs Compras)"] = pb_info.ids_transacciones_compras_bloqueantes if pb_info else "N/A"
            informe.append(detalle_t)
        return informe


    def generar_informe_perdidas_bloqueadas(self):
        informe = []
        # Incluye las que están activas y las que ya se desbloquearon (para historial)
        for pb in self.perdidas_bloqueadas_activas + self.perdidas_desbloqueadas_historial:
            informe.append({
                "ID Perdida": pb.id_perdida,
                "AssetID": pb.asset_id,
                "Fecha Venta Original": pb.fecha_venta_original.strftime('%d/%m/%Y'),
                "Importe Perdida Bloqueada EUR": f"{pb.importe_perdida_bloqueada_eur:.2f}",
                "IDs Compras Bloqueantes": pb.ids_transacciones_compras_bloqueantes,
                "Regla Aplicada (días)": pb.regla_aplicada_dias,
                "Status": pb.status,
                "Año Fiscal Generación": pb.fiscal_year_generacion,
                "Fecha Desbloqueo": pb.fecha_desbloqueo.strftime('%d/%m/%Y') if pb.fecha_desbloqueo else "N/A",
                "Año Fiscal Desbloqueo": pb.fiscal_year_desbloqueo if pb.fiscal_year_desbloqueo else "N/A",
            })
        return informe

    def generar_informe_perdidas_pendientes_compensar(self, ano_actual_declaracion):
        # Esta es una versión simplificada. Una real necesitaría rastrear el consumo de pérdidas año a año.
        informe = []
        # Primero, obtener todos los saldos netos negativos de años anteriores no prescritos
        for year in range(ano_actual_declaracion - 4, ano_actual_declaracion):
            resumen_ano_pasado = self.generar_resumen_fiscal_anual(year) # Esto podría ser costoso si no se cachean resultados
            if resumen_ano_pasado["saldo_neto_ganancias_perdidas_patrimoniales_imputables"] < 0:
                # Asumimos que no se han compensado aún para este informe simple.
                # Una implementación completa necesitaría saber cuánto se compensó cada año.
                informe.append({
                    "Año Fiscal Origen Perdida": year,
                    "Importe Original Perdida Neta": f"{resumen_ano_pasado['saldo_neto_ganancias_perdidas_patrimoniales_imputables']:.2f}",
                    "Importe Pendiente Compensar": f"{resumen_ano_pasado['saldo_neto_ganancias_perdidas_patrimoniales_imputables']:.2f} (Simplificado, asumir no compensado)",
                    "Año Fiscal Límite Compensación": year + 4
                })
        return informe

# --- Ejemplo de Uso ---
if __name__ == "__main__":
    # Crear transacciones de ejemplo (reemplazar con carga desde CSV/JSON)
    # Es crucial asignar un ID único a cada transacción si no viene en los datos.
    # Usaremos IDs simples para el ejemplo.
    transacciones_ejemplo = [
        Transaccion(id_transaccion="T0", asset_id="ACCION_X", tipo_operacion="Compra", fecha_operacion="01/01/2022", cantidad=100, precio_unitario=10, divisa="EUR", comisiones=5, tipo_cambio_eur=1, broker="BrokerA"),
        Transaccion(id_transaccion="T1", asset_id="ACCION_X", tipo_operacion="Compra", fecha_operacion="15/06/2022", cantidad=50, precio_unitario=12, divisa="EUR", comisiones=3, tipo_cambio_eur=1, broker="BrokerA"),

        # Venta con pérdida
        Transaccion(id_transaccion="T2", asset_id="ACCION_X", tipo_operacion="Venta", fecha_operacion="01/03/2023", cantidad=70, precio_unitario=8, divisa="EUR", comisiones=4, tipo_cambio_eur=1, broker="BrokerA"),
        # Recompra dentro de 2 meses (debería bloquear la pérdida de T2)
        Transaccion(id_transaccion="T3", asset_id="ACCION_X", tipo_operacion="Compra", fecha_operacion="15/03/2023", cantidad=30, precio_unitario=9, divisa="EUR", comisiones=2, tipo_cambio_eur=1, broker="BrokerA"),

        # Venta posterior que podría desbloquear la pérdida si vende T3 completamente
        Transaccion(id_transaccion="T4", asset_id="ACCION_X", tipo_operacion="Venta", fecha_operacion="01/08/2023", cantidad=30, precio_unitario=11, divisa="EUR", comisiones=2, tipo_cambio_eur=1, broker="BrokerA"), # Vende las 30 de T3

        Transaccion(id_transaccion="T5", asset_id="CRYPTO_Y", tipo_operacion="Compra", fecha_operacion="10/01/2023", cantidad=2, precio_unitario=1000, divisa="USD", comisiones=10, tipo_cambio_eur=0.9, broker="ExchangeC"),
        Transaccion(id_transaccion="T6", asset_id="CRYPTO_Y", tipo_operacion="Venta", fecha_operacion="20/05/2023", cantidad=1, precio_unitario=1500, divisa="USD", comisiones=7, tipo_cambio_eur=0.92, broker="ExchangeC"),

        # Caso: Pérdida bloqueada por compra anterior y posterior
        Transaccion(id_transaccion="T7", asset_id="ACCION_Z", tipo_operacion="Compra", fecha_operacion="01/02/2024", cantidad=100, precio_unitario=20, divisa="EUR", comisiones=10, tipo_cambio_eur=1), # Compra A
        Transaccion(id_transaccion="T8", asset_id="ACCION_Z", tipo_operacion="Venta", fecha_operacion="15/02/2024", cantidad=50, precio_unitario=15, divisa="EUR", comisiones=5, tipo_cambio_eur=1), # Venta B (con pérdida)
        Transaccion(id_transaccion="T9", asset_id="ACCION_Z", tipo_operacion="Compra", fecha_operacion="01/03/2024", cantidad=30, precio_unitario=16, divisa="EUR", comisiones=3, tipo_cambio_eur=1), # Compra C (recompra post)
        # La pérdida de T8 debería bloquearse por T7 (si se considera relevante por "antes") y T9 (por "después").
        # Según la regla "completamente vendidos los valores cuya compra originó el bloqueo",
        # la pérdida de T8 se desbloquearía cuando se vendan las acciones de T7 (la parte que no se vendió en T8) Y las de T9.
        # Mi implementación actual de bloqueo considera compras en la ventana [-X, +X] como bloqueantes.
    ]

    # Inicializar y procesar
    # Para cripto, usar regla de 2 meses (False) o 1 año (True)
    calculadora = CalculadoraFiscal(transacciones_ejemplo, config_cripto_periodo_largo=False)
    calculadora.procesar_transacciones()

    # Generar informes para un año fiscal (ej: 2023)
    ano_declaracion = 2023
    print(f"\n--- Resumen Fiscal para el año {ano_declaracion} ---")
    resumen_2023 = calculadora.generar_resumen_fiscal_anual(ano_declaracion)
    for key, value in resumen_2023.items():
        if isinstance(value, list) and key == "perdidas_pendientes_compensar_ejercicios_siguientes":
            print(f"{key}:")
            for item in value:
                print(f"  - {item}")
        elif isinstance(value, float):
             print(f"{key}: {value:.2f} EUR")
        else:
            print(f"{key}: {value}")

    ano_declaracion_2 = 2024
    print(f"\n--- Resumen Fiscal para el año {ano_declaracion_2} ---")
    resumen_2024 = calculadora.generar_resumen_fiscal_anual(ano_declaracion_2)
    for key, value in resumen_2024.items():
        if isinstance(value, list) and key == "perdidas_pendientes_compensar_ejercicios_siguientes":
            print(f"{key}:")
            for item in value:
                print(f"  - {item}")
        elif isinstance(value, float):
             print(f"{key}: {value:.2f} EUR")
        else:
            print(f"{key}: {value}")


    print("\n--- Informe Detallado para ACCION_X ---")
    informe_accion_x = calculadora.generar_informe_detallado_por_activo("ACCION_X")
    for detalle in informe_accion_x:
        print(detalle)

    print("\n--- Informe Detallado para ACCION_Z ---")
    informe_accion_z = calculadora.generar_informe_detallado_por_activo("ACCION_Z")
    for detalle in informe_accion_z:
        print(detalle)

    print("\n--- Informe de Todas las Pérdidas Bloqueadas (Activas e Historial) ---")
    informe_pb = calculadora.generar_informe_perdidas_bloqueadas()
    for pb_detalle in informe_pb:
        print(pb_detalle)

    print(f"\n--- Informe de Pérdidas Pendientes de Compensar (para declaración {ano_declaracion}) ---")
    # Este informe es muy simplificado y necesitaría una lógica más robusta de seguimiento.
    informe_ppc = calculadora.generar_informe_perdidas_pendientes_compensar(ano_declaracion)
    for ppc_detalle in informe_ppc:
        print(ppc_detalle)


--- Resumen Fiscal para el año 2023 ---
year: 2023
total_ganancias_brutas_ventas_del_ano: 495.56 EUR
total_perdidas_brutas_ventas_del_ano: -147.50 EUR
total_perdidas_bloqueadas_generadas_en_el_ano: -147.50 EUR
total_perdidas_desbloqueadas_en_el_ano_de_anos_anteriores: 0
total_perdidas_desbloqueadas_en_el_ano_generadas_en_el_mismo_ano: -147.50 EUR
saldo_neto_ganancias_perdidas_patrimoniales_imputables: 348.06 EUR
perdidas_pendientes_compensar_ejercicios_siguientes:

--- Resumen Fiscal para el año 2024 ---
year: 2024
total_ganancias_brutas_ventas_del_ano: 0
total_perdidas_brutas_ventas_del_ano: -260.00 EUR
total_perdidas_bloqueadas_generadas_en_el_ano: -260.00 EUR
total_perdidas_desbloqueadas_en_el_ano_de_anos_anteriores: 0
total_perdidas_desbloqueadas_en_el_ano_generadas_en_el_mismo_ano: 0
saldo_neto_ganancias_perdidas_patrimoniales_imputables: 0
perdidas_pendientes_compensar_ejercicios_siguientes:

--- Informe Detallado para ACCION_X ---
{'ID Transaccion': 'T0', 'Fecha': '01/01/2022',

# New Section

# New Section